In [15]:
import pandas as pd
import pprint
import numpy as np
from collections import defaultdict

### Quick look at the data

I am filtering out rows where the occupation code hasn't been recorded. Does this affect our results?

In [16]:
non_compliance_data = pd.read_csv('./data/afsa/non-compliance-in-personal-insolvencies.csv')
display(non_compliance_data.shape)
#non_compliance_data = non_compliance_data[non_compliance_data['Debtor Occupation Code (ANZSCO)'].notnull()]
display(non_compliance_data.shape)
non_compliance_data.isnull().sum()

/home/mahasen/anaconda3/envs/gh2018/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(356500, 23)

(356500, 23)

Unique ID                                0
Calendar Year of Insolvency              0
SA3 of Debtor                            0
SA3 Code of Debtor                       0
GCCSA of Debtor                          0
GCCSA Code of Debtor                     0
State of Debtor                          0
Sex of Debtor                            0
Family Situation                         0
Debtor Occupation Code (ANZSCO)      15886
Debtor Occupation Name (ANZSCO)      15886
Main Cause of Insolvency                 0
Business Related Insolvency              0
Debtor Income                            0
Primary Income Source                    0
Unsecured Debts                          0
Value of Assets                          0
Type of Party                       341629
Non-Compliance Type                 341608
Result of Non-Compliance            342306
Number of Instances                 341608
Outcome of Non-Compliance           345672
Non-Compliance Conviction Result    352942
dtype: int6

In [34]:
non_compliance_data

,Unique ID,Calendar Year of Insolvency,SA3 of Debtor,SA3 Code of Debtor,GCCSA of Debtor,GCCSA Code of Debtor,State of Debtor,Sex of Debtor,Family Situation,Debtor Occupation Code (ANZSCO),...,Primary Income Source,Unsecured Debts,Value of Assets,Type of Party,Non-Compliance Type,Result of Non-Compliance,Number of Instances,Outcome of Non-Compliance,Non-Compliance Conviction Result,Debtor Occupation Code (Major) (ANZSCO)
0,3452750,2010,South Canberra,80106,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Single with Dependants,39.0,...,Government benefits/Pensions,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,3.0
1,3563908,2011,Weston Creek,80108,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Couple without Dependants,84.0,...,Government benefits/Pensions,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,8.0
2,3252673,2012,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Single with Dependants,11.0,...,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,3610744,2013,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Couple with Dependants,99.0,...,Unknown,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,9.0
4,3610734,2013,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Couple with Dependants,22.0,...,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,2.0
5,3631198,2014,Gungahlin,80104,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Single without Dependants,36.0,...,Gross Wages and Salary,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,3.0
6,3648556,2015,Weston Creek,80108,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Couple without Dependants,59.0,...,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,5.0
7,3019574,2016,North Canberra,80105,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Single without Dependants,43.0,...,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,4.0
8,3217157,2017,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Couple with Dependants,14.0,...,Government benefits/Pensions,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,3389604,2018,South Canberra,80106,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Single without Dependants,45.0,...,Self Employment,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,4.0


### 1. Find all unique family situations in non-compliance-in-personal-insolvencies.csv

In [17]:
display(non_compliance_data['Family Situation'].unique())
family_situations = ['Single with Dependants', 'Couple without Dependants',
       'Couple with Dependants', 'Single without Dependants']

array(['Single with Dependants', 'Couple without Dependants',
       'Couple with Dependants', 'Single without Dependants', 'Unknown',
       'Not Stated'], dtype=object)

### 2. Express each such family situation in terms of the columns in B25 / G25

In [18]:
g25 = pd.read_csv('./data/census/gcpsa3/2016 Census GCP Statistical Area 3 for AUST/2016Census_G25_AUS_SA3.csv')
g25.head(5)

,SA3_CODE_2016,CF_no_children_F,CF_no_children_P,CF_ChU15_a_DSs_a_NdCh_F,CF_ChU15_a_DSs_a_NdCh_P,CF_ChU15_a_DSs_a_no_NdCh_F,CF_ChU15_a_DSs_a_no_NdCh_P,CF_ChU15_a_no_DSs_a_NdCh_F,CF_ChU15_a_no_DSs_a_NdCh_P,CF_ChU15_a_no_DSs_a_no_NdCh_F,...,OPF_no_ChU15_no_DSs_a_NdCh_F,OPF_no_ChU15_no_DSs_a_NdCh_P,OPF_no_ChU15_a_Total_F,OPF_no_ChU15_a_Total_P,OPF_Total_F,OPF_Total_P,Other_family_F,Other_family_P,Total_F,Total_P
0,10102,5577,10999,105,573,762,3495,185,844,3963,...,722,1562,994,2231,2072,5214,180,370,15092,44276
1,10103,2220,4334,29,151,180,803,34,157,1057,...,259,547,355,752,682,1712,52,103,4808,13190
2,10104,10377,20461,95,535,566,2675,204,922,2988,...,1026,2105,1387,2936,2929,7522,195,399,19216,50111
3,10105,3922,7788,70,397,345,1603,137,650,1861,...,554,1167,733,1565,1551,4039,129,263,9184,25950
4,10106,4093,8081,83,441,450,2102,126,575,2160,...,448,927,608,1317,1350,3588,118,240,9571,27619


In [19]:
persons_columns = list(filter(lambda e: e.endswith('_P'), g25.columns))
display(persons_columns)
display(len(persons_columns))

['CF_no_children_P',
 'CF_ChU15_a_DSs_a_NdCh_P',
 'CF_ChU15_a_DSs_a_no_NdCh_P',
 'CF_ChU15_a_no_DSs_a_NdCh_P',
 'CF_ChU15_a_no_DSs_a_no_NdCh_P',
 'CF_ChU15_a_Total_P',
 'CF_no_ChU15_a_DSs_a_NdCh_P',
 'CF_no_ChU15_a_DSs_a_no_NdCh_P',
 'CF_no_ChU15_a_no_DSs_a_NdCh_P',
 'CF_no_ChU15_a_Total_P',
 'CF_Total_P',
 'OPF_ChU15_a_DSs_a_NdCh_P',
 'OPF_ChU15_a_DSs_a_no_NdCh_P',
 'OPF_ChU15_a_no_DSs_a_NdCh_P',
 'OPF_ChU15_a_no_DSs_no_NdCh_P',
 'OPF_ChU15_a_Total_P',
 'OPF_no_ChU15_a_DSs_a_NdCh_P',
 'OPF_no_ChU15_a_DSs_no_NdCh_P',
 'OPF_no_ChU15_no_DSs_a_NdCh_P',
 'OPF_no_ChU15_a_Total_P',
 'OPF_Total_P',
 'Other_family_P',
 'Total_P']

23

In [23]:
a = ['CF_no_children_P', 'Couple without Dependants',
 'CF_ChU15_a_DSs_a_NdCh_P', 'Couple with Dependants',
 'CF_ChU15_a_DSs_a_no_NdCh_P', 'Couple with Dependants',
 'CF_ChU15_a_no_DSs_a_NdCh_P', 'Couple with Dependants',
 'CF_ChU15_a_no_DSs_a_no_NdCh_P', 'Couple with Dependants',
 #'CF_ChU15_a_Total_P',
 'CF_no_ChU15_a_DSs_a_NdCh_P', 'Couple with Dependants',
 'CF_no_ChU15_a_DSs_a_no_NdCh_P', 'Couple with Dependants',
 'CF_no_ChU15_a_no_DSs_a_NdCh_P', 'Couple without Dependants', 
 #'CF_no_ChU15_a_Total_P',
 #'CF_Total_P',
 'OPF_ChU15_a_DSs_a_NdCh_P', 'Single with Dependants',
 'OPF_ChU15_a_DSs_a_no_NdCh_P', 'Single with Dependants',
 'OPF_ChU15_a_no_DSs_a_NdCh_P', 'Single with Dependants',
 'OPF_ChU15_a_no_DSs_no_NdCh_P', 'Single with Dependants',
 #'OPF_ChU15_a_Total_P',
 'OPF_no_ChU15_a_DSs_a_NdCh_P', 'Single with Dependants',
 'OPF_no_ChU15_a_DSs_no_NdCh_P', 'Single with Dependants',
 'OPF_no_ChU15_no_DSs_a_NdCh_P', 'Single without Dependants',
 #'OPF_no_ChU15_a_Total_P',
 #'OPF_Total_P',
 #'Other_family_P',
#'Total_P'
    ]

In [24]:
b25_to_insolvencies = dict(zip(a[::2], a[1::2]))

insolvencies_to_b25 = defaultdict(list)
for a,b in (zip(a[1::2], a[::2])):
    insolvencies_to_b25[a].append(b)
#insolvencies_to_b25 = dict(zip(a[1::2], a[::2]))
display(b25_to_insolvencies)
display(insolvencies_to_b25)

{'CF_no_children_P': 'Couple without Dependants',
 'CF_ChU15_a_DSs_a_NdCh_P': 'Couple with Dependants',
 'CF_ChU15_a_DSs_a_no_NdCh_P': 'Couple with Dependants',
 'CF_ChU15_a_no_DSs_a_NdCh_P': 'Couple with Dependants',
 'CF_ChU15_a_no_DSs_a_no_NdCh_P': 'Couple with Dependants',
 'CF_no_ChU15_a_DSs_a_NdCh_P': 'Couple with Dependants',
 'CF_no_ChU15_a_DSs_a_no_NdCh_P': 'Couple with Dependants',
 'CF_no_ChU15_a_no_DSs_a_NdCh_P': 'Couple without Dependants',
 'OPF_ChU15_a_DSs_a_NdCh_P': 'Single with Dependants',
 'OPF_ChU15_a_DSs_a_no_NdCh_P': 'Single with Dependants',
 'OPF_ChU15_a_no_DSs_a_NdCh_P': 'Single with Dependants',
 'OPF_ChU15_a_no_DSs_no_NdCh_P': 'Single with Dependants',
 'OPF_no_ChU15_a_DSs_a_NdCh_P': 'Single with Dependants',
 'OPF_no_ChU15_a_DSs_no_NdCh_P': 'Single with Dependants',
 'OPF_no_ChU15_no_DSs_a_NdCh_P': 'Single without Dependants'}

defaultdict(list,
            {'Couple without Dependants': ['CF_no_children_P',
              'CF_no_ChU15_a_no_DSs_a_NdCh_P'],
             'Couple with Dependants': ['CF_ChU15_a_DSs_a_NdCh_P',
              'CF_ChU15_a_DSs_a_no_NdCh_P',
              'CF_ChU15_a_no_DSs_a_NdCh_P',
              'CF_ChU15_a_no_DSs_a_no_NdCh_P',
              'CF_no_ChU15_a_DSs_a_NdCh_P',
              'CF_no_ChU15_a_DSs_a_no_NdCh_P'],
             'Single with Dependants': ['OPF_ChU15_a_DSs_a_NdCh_P',
              'OPF_ChU15_a_DSs_a_no_NdCh_P',
              'OPF_ChU15_a_no_DSs_a_NdCh_P',
              'OPF_ChU15_a_no_DSs_no_NdCh_P',
              'OPF_no_ChU15_a_DSs_a_NdCh_P',
              'OPF_no_ChU15_a_DSs_no_NdCh_P'],
             'Single without Dependants': ['OPF_no_ChU15_no_DSs_a_NdCh_P']})

You can extract the data like this:

In [25]:
x = g25.loc[0]
#display(x)
x[['CF_no_children_P', 'CF_ChU15_a_DSs_a_NdCh_P']]
x[insolvencies_to_b25['Couple without Dependants']].sum()
# Apparently you can do this with .groupby

14915

### 3. Produce a new version of B25 whose columns are the family situations found in (1)

In [26]:
new_columns = ['SA3_CODE_2016', *family_situations]
def g25_to_insolvencies(row):
    data = [row['SA3_CODE_2016']]
    data.extend([row[insolvencies_to_b25[x]].sum() for x in family_situations])
    return data

yes_finally_why_was_this_so_hard = g25.apply(g25_to_insolvencies, axis=1, result_type='expand')
yes_finally_why_was_this_so_hard.columns = new_columns
display(yes_finally_why_was_this_so_hard.head(5))
yes_finally_why_was_this_so_hard.to_csv('./data/preprocess/b25_but_with_less_family_situation_granularity.csv', index=False)

,SA3_CODE_2016,Single with Dependants,Couple without Dependants,Couple with Dependants,Single without Dependants
0,10102,3658,14915,23770,1562
1,10103,1164,5292,6082,547
2,10104,5422,24122,18069,2105
3,10105,2863,10250,11383,1167
4,10106,2660,10354,13446,927


### Add sub-major group column to non-compliance data

In [27]:
non_compliance_data['Debtor Occupation Code (Major) (ANZSCO)'] = non_compliance_data['Debtor Occupation Code (ANZSCO)'].apply(lambda e: np.nan if np.isnan(e) else int(str(e)[0])).fillna(10.0)
non_compliance_data.to_csv('./data/preprocess/non-compliance-major-group.csv', index=False)
non_compliance_data.head(10)

,Unique ID,Calendar Year of Insolvency,SA3 of Debtor,SA3 Code of Debtor,GCCSA of Debtor,GCCSA Code of Debtor,State of Debtor,Sex of Debtor,Family Situation,Debtor Occupation Code (ANZSCO),...,Primary Income Source,Unsecured Debts,Value of Assets,Type of Party,Non-Compliance Type,Result of Non-Compliance,Number of Instances,Outcome of Non-Compliance,Non-Compliance Conviction Result,Debtor Occupation Code (Major) (ANZSCO)
0,3452750,2010,South Canberra,80106,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Single with Dependants,39.0,...,Government benefits/Pensions,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,3.0
1,3563908,2011,Weston Creek,80108,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Couple without Dependants,84.0,...,Government benefits/Pensions,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,8.0
2,3252673,2012,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Single with Dependants,11.0,...,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,3610744,2013,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Couple with Dependants,99.0,...,Unknown,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,9.0
4,3610734,2013,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Couple with Dependants,22.0,...,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,2.0
5,3631198,2014,Gungahlin,80104,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Single without Dependants,36.0,...,Gross Wages and Salary,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,3.0
6,3648556,2015,Weston Creek,80108,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Couple without Dependants,59.0,...,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,5.0
7,3019574,2016,North Canberra,80105,Australian Capital Territory,8ACTE,Australian Capital Territory,Male,Single without Dependants,43.0,...,Gross Wages and Salary,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,4.0
8,3217157,2017,Tuggeranong,80107,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Couple with Dependants,14.0,...,Government benefits/Pensions,$0-$49999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,3389604,2018,South Canberra,80106,Australian Capital Territory,8ACTE,Australian Capital Territory,Female,Single without Dependants,45.0,...,Self Employment,$50000-$99999,$0-$49999,NaN,NaN,NaN,NaN,NaN,NaN,4.0


### Pull in G57A Occupation

In [11]:
g57 = pd.read_csv('./data/census/gcpsa3/2016 Census GCP Statistical Area 3 for AUST/2016Census_G57A_AUS_SA3.csv')
#display(g57.head(10))
#g57.columns.format()

keys_of_interest = [
    'SA3_CODE_2016',
 'M_Tot_Managers',
 'M_Tot_Professionals',
 'M_Tot_TechnicTrades_W',
 'M_Tot_CommunPersnlSvc_W',
 'M_Tot_ClericalAdminis_W',
 'M_Tot_Sales_W',
 'M_Tot_Mach_oper_drivers',
 'M_Tot_Labourers',
 'M_Tot_Occu_ID_NS',
 'M_Tot_Tot',
 'F_Tot_Managers',
 'F_Tot_Professionals',
 'F_Tot_TechnicTrades_W',
 'F_Tot_CommunPersnlSvc_W',
 'F_Tot_ClericalAdminis_W',
 'F_Tot_Sales_W',
 'F_Tot_Mach_oper_drivers',
 'F_Tot_Labourers',
 'F_Tot_Occu_ID_NS',
 'F_Tot_Tot',]

In [33]:
g57_interest = g57[keys_of_interest]
import itertools
cols = ['SA3_CODE_2016']
for n in map(float,range(1,10)):
    cols.append("M_{}".format(np.int(n)))
cols.append("M_Tot_Tot")
for n in map(float,range(1,10)):
    cols.append("F_{}".format(np.int(n)))
cols.append("F_Tot_Tot")
g57_interest.columns =  cols
g57_interest.head(10)

,SA3_CODE_2016,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,M_9,...,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,F_9,F_Tot_Tot
0,10102,2900,2586,3454,1112,1370,954,1307,1533,361,...,1964,3382,592,1973,4102,1396,86,631,317,14451
1,10103,1115,475,1118,377,172,341,563,723,107,...,660,802,199,778,892,556,58,422,66,4441
2,10104,2045,1526,3388,913,528,1052,1362,2145,255,...,1341,2655,735,2416,2524,1884,110,1142,177,12977
3,10105,1459,757,1851,786,340,540,1112,1170,157,...,765,1432,346,1462,1471,875,125,644,105,7231
4,10106,2081,966,1819,410,436,478,834,1285,174,...,1222,1705,370,1224,1678,784,64,620,124,7789
5,10201,5834,7132,9121,2767,2351,2858,2999,4540,753,...,3305,9241,1642,6079,8204,4593,377,2162,498,36102
6,10202,3991,3504,9245,2335,1693,2504,4482,5305,668,...,2515,6025,1580,6131,6840,4899,499,2446,468,31413
7,10301,1654,1300,2481,887,437,664,1375,1459,211,...,934,2399,466,1764,1973,1273,102,885,138,9942
8,10302,3203,895,2551,446,335,686,1748,1829,260,...,1504,2088,483,1748,1921,1205,161,959,169,10230
9,10303,1429,750,2596,475,309,482,2094,1310,179,...,863,1531,484,1521,1741,1087,131,874,114,8339


# Bayesian predictor
Let's build two predictors:
- one using B25 for priors and non-compliance for posteriors (SA3 and Family Situation)
- one usng G57A for priors and non-compliance for posteriors (SA3, Sex, Major ANZSCO Occupation Code)

We then need to find a way to combine these predictors to give a single number. Adding in quadrature after normalising by the non-compliant marginal probability seems relatively safe.

Then let's calculate the average expected risk of non-compliance (i.e. the marginal risk of non-compliance), and express every prediction in units of this quantity (i.e. person X is 2.5x more likely to be non-compliant than average)

In [74]:
b25 = yes_finally_why_was_this_so_hard.set_index('SA3_CODE_2016')
g57a = g57_interest.set_index('SA3_CODE_2016')
non_comp = non_compliance_data[['SA3 Code of Debtor','Sex of Debtor','Family Situation','Debtor Occupation Code (Major) (ANZSCO)']]

In [76]:
# make list of SA3 codes
SA3_CODE_2016_list = list(set(non_comp['SA3 Code of Debtor'].values).union(set(b25.index.values)).union(set(g57a.index.values)))
family_situations


['Single with Dependants',
 'Couple without Dependants',
 'Couple with Dependants',
 'Single without Dependants']

In [128]:
# Make dict for non_comp priors for b25
b25_marginal = {} # marginal probability of being both in a particular SA3 and FS
b25_prior = {}    # probability of being in a particular SA3 and FS given non-compliance
b25_posterior = {}# probability of being non-compliant given someone in a particular SA3 and FS

b25_total_pop = b25.values.sum()
non_comp_pop = len(non_comp)
non_comp_marginal = np.divide(non_comp_pop,b25_total_pop) # i.e.  marginal probability of non-compliance. Note that this depends on b25!
for sa3 in SA3_CODE_2016_list:
    for fs in family_situations:
        # Calculate marginal distribution
        if sa3 in b25.index.values:
            b25_marginal[(sa3,fs)] = np.divide(b25[fs][sa3],b25_total_pop) # i.e. the probability of being both in a particular SA3 and FS
        else:
            b25_marginal[(sa3,fs)] = 0
        
        # Calculate prior
        b25_prior[(sa3,fs)] = np.divide(len(non_comp[(non_comp['SA3 Code of Debtor']==sa3) & (non_comp['Family Situation'] == fs)]),non_comp_pop)
        
        # Calculate posterior
        if b25_marginal[(sa3,fs)]>0:
            b25_posterior[(sa3,fs)] = b25_prior[(sa3,fs)] * non_comp_marginal / b25_marginal[(sa3,fs)]
        else:
            b25_posterior[(sa3,fs)] = 0  

In [134]:
# Make dict for non_comp priors for b25
g57a_marginal = {} # marginal probability of being in a particular SA3, in a particular Major ANZSCOS group, with a particular gender
g57a_prior = {}    # probability of (SA3,Sex,ANZCOS major group) given non-compliance
g57a_posterior = {} # probability of being non-compliant given someone with (SA3,Sex,ANZCOS major group)

g57a_total_pop = g57a[['M_Tot_Tot','F_Tot_Tot']].values.sum()
non_comp_pop = len(non_comp)
non_comp_marginal_g57a = np.divide(non_comp_pop,g57a_total_pop) # i.e.  marginal probability of non-compliance. Note that this depends on g57a!
for sa3 in SA3_CODE_2016_list:
    for sex in ['Male','Female']:
        for major_group in range(1,10):
            # Calculate marginal distribution
            if sa3 in g57a.index.values:
                g57a_marginal[(sa3,sex,major_group)] = np.divide(g57a['%s_%d'%(sex[0],major_group)][sa3],g57a_total_pop)
            else:
                g57a_marginal[(sa3,sex,major_group)] = 0
        
            # Calculate prior
            g57a_prior[(sa3,sex,major_group)] = np.divide(len(non_comp[(non_comp['SA3 Code of Debtor']==sa3) & (non_comp['Sex of Debtor'] == sex) & (non_comp['Debtor Occupation Code (Major) (ANZSCO)']==major_group)]),non_comp_pop)
            
            # Calculate posterior
            if g57a_marginal[(sa3,sex,major_group)] > 0:
                g57a_posterior[(sa3,sex,major_group)] = g57a_prior[(sa3,sex,major_group)] * non_comp_marginal_g57a / g57a_marginal[(sa3,sex,major_group)]
            else:
                g57a_posterior[(sa3,sex,major_group)] = 0

In [135]:
# Let's save everything
import pickle
with open("bayesian_models.pickle", "wb") as file:
    pickle.dump( {'b25_marginal':b25_marginal,
                  'b25_prior':b25_prior,
                  'b25_posterior':b25_posterior,
                  'non_comp_marginal':non_comp_marginal,
                  'b25_total_pop':b25_total_pop,
                  'g57a_marginal':g57a_marginal,
                  'g57a_prior':g57a_prior,
                  'g57a_posterior':g57a_posterior,
                  'g57a_total_pop':g57a_total_pop,
                  'non_comp_marginal_g57a':non_comp_marginal_g57a,
                  'non_comp_pop':non_comp_pop,
                  'b25':b25,
                  'g57a':g57a, 
                  'non_comp':non_comp}, file)

In [ ]:
def calculate_risk(sa3,sex,major_group,family_situation):
        risk_factor_b25 = b25_posterior[(sa3,family_situation)]/non_comp_marginal
        risk_factor_g57a= g57a_posterior[(sa3,sex,major_group)]/non_comp_marginal_g57a
        
        return sqrt(risk_factor_b25)
                                         

In [136]:
23***4

4

In [108]:
b25_posterior[(10102,'Single with Dependants')]/non_comp_marginal

1.5033139423515636

In [ ]:
non_comp[non_comp['Debtor Occupation Code (Major) (ANZSCO)']==3]

In [121]:
tmp = 'Male'

In [123]:
non_comp

,SA3 Code of Debtor,Sex of Debtor,Family Situation,Debtor Occupation Code (Major) (ANZSCO)
0,80106,Female,Single with Dependants,3.0
1,80108,Male,Couple without Dependants,8.0
2,80107,Female,Single with Dependants,1.0
3,80107,Female,Couple with Dependants,9.0
4,80107,Male,Couple with Dependants,2.0
5,80104,Male,Single without Dependants,3.0
6,80108,Female,Couple without Dependants,5.0
7,80105,Male,Single without Dependants,4.0
8,80107,Female,Couple with Dependants,1.0
9,80106,Female,Single without Dependants,4.0


In [109]:
g57a

,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,M_9,M_Tot_Tot,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,F_9,F_Tot_Tot
SA3_CODE_2016,,,,,,,,,,,,,,,,,,,,
10102,2900,2586,3454,1112,1370,954,1307,1533,361,15574,1964,3382,592,1973,4102,1396,86,631,317,14451
10103,1115,475,1118,377,172,341,563,723,107,4993,660,802,199,778,892,556,58,422,66,4441
10104,2045,1526,3388,913,528,1052,1362,2145,255,13213,1341,2655,735,2416,2524,1884,110,1142,177,12977
10105,1459,757,1851,786,340,540,1112,1170,157,8182,765,1432,346,1462,1471,875,125,644,105,7231
10106,2081,966,1819,410,436,478,834,1285,174,8485,1222,1705,370,1224,1678,784,64,620,124,7789
10201,5834,7132,9121,2767,2351,2858,2999,4540,753,38355,3305,9241,1642,6079,8204,4593,377,2162,498,36102
10202,3991,3504,9245,2335,1693,2504,4482,5305,668,33731,2515,6025,1580,6131,6840,4899,499,2446,468,31413
10301,1654,1300,2481,887,437,664,1375,1459,211,10467,934,2399,466,1764,1973,1273,102,885,138,9942
10302,3203,895,2551,446,335,686,1748,1829,260,11957,1504,2088,483,1748,1921,1205,161,959,169,10230
